# 03 - Transfer Feature Alerts Unit

**Purpose**: Generate alerts for newly activated or high-risk preview features

**Inputs**:
- Delta Table: `preview_features_active`
- Delta Table: `feature_alerts` (for historical tracking)

**Outputs**:
- Delta Table: `feature_alerts`

**Frequency**: Daily (recommended)

**Alert Triggers**:
- New preview feature detected
- Preview feature active >90 days (approaching GA)
- Preview feature with low confidence match (similarity <0.5)

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta

## Parameters

In [ ]:
# Alert thresholds
ALERT_DAYS_THRESHOLD = 90  # Alert if preview active >90 days
LOW_CONFIDENCE_THRESHOLD = 0.5  # Alert if similarity score <0.5

# Alert severity levels
SEVERITY_INFO = "Info"
SEVERITY_WARNING = "Warning"
SEVERITY_CRITICAL = "Critical"

## Step 1: Load Current Activated Preview Features

In [ ]:
print("🔄 Loading activated preview features...")

df_active = spark.read.format("delta").load("Tables/preview_features_active")

active_count = df_active.count()
print(f"✅ Loaded {active_count} activated preview features")

df_active.select("feature_id", "feature_name", "workload", "days_since_release", "similarity_score").show(5, truncate=False)

## Step 2: Load Historical Alerts (to avoid duplicates)

In [ ]:
table_name = "feature_alerts"
table_path = f"Tables/{table_name}"

try:
    from delta.tables import DeltaTable
    
    if DeltaTable.isDeltaTable(spark, table_path):
        print("🔄 Loading historical alerts...")
        df_historical = spark.read.format("delta").load(table_path)
        historical_count = df_historical.count()
        print(f"✅ Loaded {historical_count} historical alerts")
        
        # Get already alerted feature_ids
        alerted_features = set([row["feature_id"] for row in df_historical.select("feature_id").distinct().collect()])
        print(f"  → {len(alerted_features)} unique features already alerted")
    else:
        print("ℹ️ No historical alerts found (first run)")
        df_historical = None
        alerted_features = set()
except Exception as e:
    print(f"ℹ️ No historical alerts (first run): {e}")
    df_historical = None
    alerted_features = set()

## Step 3: Generate Alerts

In [ ]:
def generate_alerts(df_active, alerted_features):
    """
    Generate alerts based on business rules
    Returns list of alert dicts
    """
    alerts = []
    
    for row in df_active.collect():
        feature_id = row["feature_id"]
        feature_name = row["feature_name"]
        workload = row["workload"]
        days_since_release = row["days_since_release"]
        similarity_score = row["similarity_score"]
        setting_name = row["setting_name"]
        
        # Rule 1: NEW PREVIEW FEATURE ACTIVATED
        if feature_id not in alerted_features:
            alerts.append({
                "alert_id": f"NEW_{feature_id}_{datetime.now().strftime('%Y%m%d')}",
                "feature_id": feature_id,
                "feature_name": feature_name,
                "workload": workload,
                "alert_type": "New Preview Activated",
                "severity": SEVERITY_INFO,
                "message": f"New preview feature '{feature_name}' has been activated (Setting: {setting_name})",
                "days_since_release": days_since_release,
                "similarity_score": similarity_score,
                "alert_date": datetime.now(),
                "acknowledged": False
            })
        
        # Rule 2: LONG-RUNNING PREVIEW (>90 days)
        if days_since_release and days_since_release > ALERT_DAYS_THRESHOLD:
            alert_id = f"LONGRUN_{feature_id}_{datetime.now().strftime('%Y%m%d')}"
            
            alerts.append({
                "alert_id": alert_id,
                "feature_id": feature_id,
                "feature_name": feature_name,
                "workload": workload,
                "alert_type": "Long-Running Preview",
                "severity": SEVERITY_WARNING,
                "message": f"Preview feature '{feature_name}' has been active for {days_since_release} days. Consider reviewing for GA transition.",
                "days_since_release": days_since_release,
                "similarity_score": similarity_score,
                "alert_date": datetime.now(),
                "acknowledged": False
            })
        
        # Rule 3: LOW CONFIDENCE MATCH
        if similarity_score < LOW_CONFIDENCE_THRESHOLD:
            alerts.append({
                "alert_id": f"LOWCONF_{feature_id}_{datetime.now().strftime('%Y%m%d')}",
                "feature_id": feature_id,
                "feature_name": feature_name,
                "workload": workload,
                "alert_type": "Low Confidence Match",
                "severity": SEVERITY_CRITICAL,
                "message": f"Low confidence match ({similarity_score:.2f}) between setting '{setting_name}' and feature '{feature_name}'. Manual review recommended.",
                "days_since_release": days_since_release,
                "similarity_score": similarity_score,
                "alert_date": datetime.now(),
                "acknowledged": False
            })
    
    return alerts

In [ ]:
print("🔄 Generating alerts based on business rules...")

alerts_data = generate_alerts(df_active, alerted_features)

print(f"✅ Generated {len(alerts_data)} new alerts")

# Preview alerts
if alerts_data:
    print("\n📋 Sample alerts:")
    for alert in alerts_data[:5]:
        print(f"  [{alert['severity']}] {alert['alert_type']}")
        print(f"    → {alert['message']}")
        print()

## Step 4: Create Alerts DataFrame

In [ ]:
if alerts_data:
    schema = StructType([
        StructField("alert_id", StringType(), False),
        StructField("feature_id", StringType(), False),
        StructField("feature_name", StringType(), False),
        StructField("workload", StringType(), True),
        StructField("alert_type", StringType(), False),
        StructField("severity", StringType(), False),
        StructField("message", StringType(), False),
        StructField("days_since_release", IntegerType(), True),
        StructField("similarity_score", DoubleType(), True),
        StructField("alert_date", TimestampType(), False),
        StructField("acknowledged", BooleanType(), False)
    ])
    
    df_alerts = spark.createDataFrame(alerts_data, schema=schema)
    
    print(f"✅ Created alerts DataFrame with {df_alerts.count()} rows")
    df_alerts.show(5, truncate=False)
else:
    print("ℹ️ No new alerts to generate")
    df_alerts = None

## Step 5: Write Alerts to Delta Lake

In [ ]:
if df_alerts and df_alerts.count() > 0:
    print(f"🔄 Writing alerts to Delta table: {table_name}")
    
    try:
        from delta.tables import DeltaTable
        
        # Check if table exists
        if DeltaTable.isDeltaTable(spark, table_path):
            print("  → Table exists, appending new alerts...")
            
            # Append mode - keep all alerts for historical tracking
            df_alerts.write.format("delta").mode("append").save(table_path)
            
            print("  ✅ Alerts appended")
        else:
            print("  → Table doesn't exist, creating new table...")
            df_alerts.write.format("delta").mode("overwrite").save(table_path)
            print("  ✅ Table created")
        
        # Show final count
        final_count = spark.read.format("delta").load(table_path).count()
        print(f"\n✅ Total alerts in {table_name}: {final_count}")
        
    except Exception as e:
        print(f"❌ Error writing alerts: {e}")
        raise
else:
    print("ℹ️ No alerts to write")

## Step 6: Alert Summary Statistics

In [ ]:
from delta.tables import DeltaTable

if DeltaTable.isDeltaTable(spark, table_path):
    print("\n📊 Alert Statistics:")
    print("=" * 60)
    
    df_all_alerts = spark.read.format("delta").load(table_path)
    
    # By severity
    print("\n🔸 By Severity:")
    df_all_alerts.groupBy("severity").count().orderBy(F.desc("count")).show(truncate=False)
    
    # By alert type
    print("\n🔸 By Alert Type:")
    df_all_alerts.groupBy("alert_type").count().orderBy(F.desc("count")).show(truncate=False)
    
    # By workload
    print("\n🔸 By Workload:")
    df_all_alerts.groupBy("workload").count().orderBy(F.desc("count")).show(truncate=False)
    
    # Unacknowledged alerts
    unack_count = df_all_alerts.filter(F.col("acknowledged") == False).count()
    print(f"\n🔸 Unacknowledged alerts: {unack_count}")
    
    # Recent alerts (last 7 days)
    recent_cutoff = datetime.now() - timedelta(days=7)
    recent_count = df_all_alerts.filter(F.col("alert_date") >= recent_cutoff).count()
    print(f"🔸 Alerts in last 7 days: {recent_count}")
    
    # Critical alerts
    critical_count = df_all_alerts.filter(F.col("severity") == SEVERITY_CRITICAL).count()
    print(f"🔸 Critical alerts: {critical_count}")
    
    print("\n" + "=" * 60)

print("✅ Transfer Feature Alerts Unit - COMPLETED")

## Step 7: Export Alerts for Data Activator (Optional)

This creates a view that Data Activator can monitor

In [ ]:
from delta.tables import DeltaTable

if DeltaTable.isDeltaTable(spark, table_path):
    print("\n🔄 Creating view for Data Activator integration...")
    
    df_activator = spark.read.format("delta").load(table_path)
    
    # Filter to unacknowledged critical/warning alerts
    df_activator_filtered = df_activator.filter(
        (F.col("acknowledged") == False) & 
        (F.col("severity").isin([SEVERITY_CRITICAL, SEVERITY_WARNING]))
    )
    
    # Create or replace temp view
    df_activator_filtered.createOrReplaceTempView("vw_feature_alerts_active")
    
    alert_count = df_activator_filtered.count()
    print(f"✅ Created view 'vw_feature_alerts_active' with {alert_count} active alerts")
    print("   → This view can be monitored by Data Activator for real-time notifications")